In [78]:
#Import necessary libraries
import json
import pandas as pd
import numpy as np

In [79]:
#Reading data from s3
df = pd.read_json("https://raw.githubusercontent.com/ajaykuma/Datasets_For_Work/refs/heads/main/recipes.json",lines=True)

#creating a copy of df
Workdf = df.copy()

#Looking at columns
Workdf.columns

Index(['name', 'ingredients', 'url', 'image', 'cookTime', 'recipeYield',
       'datePublished', 'prepTime', 'description'],
      dtype='object')

In [4]:
Workdf.head(2)

,name,ingredients,url,image,cookTime,recipeYield,datePublished,prepTime,description
0,Easter Leftover Sandwich,12 whole Hard Boiled Eggs\n1/2 cup Mayonnaise\...,http://thepioneerwoman.com/cooking/2013/04/eas...,http://static.thepioneerwoman.com/cooking/file...,PT,8,2013-04-01,PT15M,Got leftover Easter eggs? Got leftover East...
1,Pasta with Pesto Cream Sauce,3/4 cups Fresh Basil Leaves\n1/2 cup Grated Pa...,http://thepioneerwoman.com/cooking/2011/06/pas...,http://static.thepioneerwoman.com/cooking/file...,PT10M,8,2011-06-06,PT6M,I finally have basil in my garden. Basil I can...


In [80]:
#Function to extract recipes that contain beef
def findingbeef(x:str):
    if 'beef' in x:
        return x
    elif 'Beef' in x:
        return x
    else:
        return 'not beef'

In [81]:
#extracting rows with recipes without beef
Workdf['ingredients'] = Workdf['ingredients'].apply(lambda n:findingbeef(n))
Workdf = Workdf[Workdf.ingredients != 'not beef'] 
Workdf.count()

name             47
ingredients      47
url              47
image            47
cookTime         47
recipeYield      47
datePublished    47
prepTime         47
description      47
dtype: int64

In [82]:
#looking at unique values
Workdf['cookTime'].unique()
Workdf['prepTime'].unique()

array(['PT10M', 'PT20M', 'PT5M', 'PT15M', 'PT25M', 'PT1M', 'PT40M',
       'PT30M', 'PT1H30M', 'PT45M', 'PT2H'], dtype=object)

In [83]:
#replacing 'PT' and '' values with PT0
#if there are missing values or no time duration (as it exists in the dataset)
#Workdf['cookTime'] = Workdf['cookTime'].replace(['PT',''],'PT0')
#Workdf['prepTime'] = Workdf['prepTime'].replace(['PT',''],'PT0')

#we dont need for our use case
Workdf['prepTime'] = Workdf['prepTime'].apply(lambda n:n[2:])
Workdf['cookTime'] = Workdf['cookTime'].apply(lambda n:n[2:])

In [26]:
#Workdf['prepTime']
Workdf[['cookTime','prepTime']].head(10)

,cookTime,prepTime
7,25M,10M
25,3H,20M
27,25M,5M
36,20M,10M
68,4H,5M
70,25M,20M
71,1H,20M
74,10M,5M
77,15M,5M
104,3H,10M


In [84]:
#using timedelta to convert 
def timeff(x):
    if x[1] == 'H':
        x = x[0]+x[1].replace('H','h')
    elif x[1] == 'M':
        x = x[0]+x[1].replace('S','s')
    elif x[1] == 'S':
        x = x[0]+x[1].replace('S','s')
    return x
    n = pd.Timedelta(x)
    return n

In [77]:
#pd.Timedelta('1min')
#Workdf['cookTime'][7].apply(lambda n:timeff(pd.Timedelta(n)))

AttributeError: 'str' object has no attribute 'apply'

In [85]:
#before applying fucntion,confirming datatypes
Workdf.dtypes

name             object
ingredients      object
url              object
image            object
cookTime         object
recipeYield      object
datePublished    object
prepTime         object
description      object
dtype: object

In [63]:
Workdf[['name','prepTime','cookTime']]

,name,prepTime,cookTime
7,Patty Melts,10M,25M
25,Spicy Stewed Beef with Creamy Cheddar Grits,20M,3h
27,Pork Chops with Garlic and Wine,5min,25M
36,"Salisbury Steak, Mashed Potatoes, and Peas",10M,20M
68,Hot &amp; Spicy Italian Drip Beef,5min,4h
70,Surf &amp; Turf Cajun Pasta,20M,25M
71,Spaghetti Sauce,20M,1h
74,Pepperoni Pizza Burgers,5min,10M
77,Supreme Pizza Burgers,5min,15M
104,Short Ribs with Wine and Cream,10M,3h


In [86]:
#applying function
Workdf['cookTime'] = Workdf['cookTime'].apply(lambda n:timeff(n))
Workdf['prepTime'] = Workdf['prepTime'].apply(lambda n:timeff(n))

In [87]:
Workdf[['name','prepTime','cookTime']].head(5)

,name,prepTime,cookTime
7,Patty Melts,10M,25M
25,Spicy Stewed Beef with Creamy Cheddar Grits,20M,3h
27,Pork Chops with Garlic and Wine,5M,25M
36,"Salisbury Steak, Mashed Potatoes, and Peas",10M,20M
68,Hot &amp; Spicy Italian Drip Beef,5M,4h


In [88]:
type(Workdf['prepTime'][7])

str

In [91]:
#to be checked
#converting values in cookTime n PrepTime into minutes
#Workdf['cookTime'] = Workdf['cookTime']/np.timedelta64(1, 'm')
#Workdf['prepTime'] = Workdf['prepTime']/np.timedelta64(1, 'm')

In [15]:
#function to assess difficulty level
def difflvl(x):
    if x<30:
        return 'easy'
    elif x>=30 and x<=60:
        return 'medium'
    else:
        return 'hard'

In [16]:
#computing total cook time and applying function
Workdf['cookTimeTot'] = Workdf['cookTime']+Workdf['prepTime']
Workdf['difficulty'] = Workdf['cookTimeTot'].apply(lambda n:difflvl(n))

C:\Users\Ajay\AppData\Local\Temp\ipykernel_8256\1328425785.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Workdf['cookTimeTot'] = Workdf['cookTime']+Workdf['prepTime']


TypeError: '<' not supported between instances of 'str' and 'int'

In [13]:
#Groupby and fining avg cookTime for each level of difficulty
Result = Workdf.groupby(['difficulty']).agg({'cookTime':['mean']})
Result.columns = Result.columns.get_level_values(1)
Result = Result.reset_index()

In [14]:
#Result['difficulty'] = Result['difflvl']
Result['avg_total_cooking_time'] = Result['mean']
#Result.drop(['mean','difflvl'],axis=1,inplace=True)
Result.drop(['mean'],axis=1,inplace=True)
#Result.to_csv('report.csv',index=False)

In [15]:
Result

,difficulty,avg_total_cooking_time
0,easy,9.625000
1,hard,169.782609
2,medium,28.125000
